Step 1: Import Required Libraries

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

Step 2: Load Input Files

In [29]:
# Load the input CSV files
credit_df = pd.read_csv("data_set/credit_record.csv")
application_df = pd.read_csv("data_set/application_record.csv")

print("Credit Record Data (First 5 rows):")
print(credit_df.head())

print("\nApplication Record Data (First 5 rows):")
print(application_df.head())

Credit Record Data (First 5 rows):
        ID  MONTHS_BALANCE STATUS
0  5001711               0      X
1  5001711              -1      0
2  5001711              -2      0
3  5001711              -3      0
4  5001712               0      C

Application Record Data (First 5 rows):
        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008809           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3

Step 3: Remove Exact Duplicates in application_record.csv

In [30]:
# Remove exact duplicate rows (where all attributes are the same)
application_df = application_df.drop_duplicates()

print(f"After removing duplicates, the application data has {len(application_df)} records.")

After removing duplicates, the application data has 438557 records.


Step 4: Sort Data by ID

In [31]:
# Sort both datasets by 'ID'
credit_df = credit_df.sort_values(by="ID")
application_df = application_df.sort_values(by="ID")

Step 5: Replace Values in application_record.csv

In [32]:
# Replace 'Y' with 1 and 'N' with 0 in FLAG_OWN_CAR and FLAG_OWN_REALTY
application_df["FLAG_OWN_CAR"] = application_df["FLAG_OWN_CAR"].replace({"Y": 1, "N": 0})
application_df["FLAG_OWN_REALTY"] = application_df["FLAG_OWN_REALTY"].replace({"Y": 1, "N": 0})

Step 6: Add Constant Column

In [33]:
# Add a fixed date column
application_df["Current_Date"] = pd.to_datetime("2022-01-01")

Step 7: Calculate Age and Years of Working

In [34]:
# Calculate Applicant Age and Years of Working
application_df["Applicant_Age"] = -application_df["DAYS_BIRTH"] // 365
application_df["Years_of_Working"] = -application_df["DAYS_EMPLOYED"] // 365

Step 8: Filter Application Data

In [35]:
# Apply filtering conditions
filtered_application_df = application_df[
    (application_df["Applicant_Age"] >= 21) &
    application_df["CODE_GENDER"].notnull() &
    application_df["FLAG_OWN_CAR"].notnull() &
    application_df["FLAG_OWN_REALTY"].notnull() &
    application_df["CNT_CHILDREN"].notnull() &
    application_df["AMT_INCOME_TOTAL"].notnull() &
    application_df["NAME_INCOME_TYPE"].notnull() &
    application_df["NAME_EDUCATION_TYPE"].notnull() &
    application_df["NAME_FAMILY_STATUS"].notnull() &
    application_df["NAME_HOUSING_TYPE"].notnull()
    ]

Step 9: Process credit_record.csv

In [36]:
# Map STATUS to Good Debt and Bad Debt
status_mapping = {"C": "Good Debt", "X": "Good Debt", "0": "Good Debt",
                  "1": "Bad Debt", "2": "Bad Debt", "3": "Bad Debt",
                  "4": "Bad Debt", "5": "Bad Debt"}
credit_df["STATUS2"] = credit_df["STATUS"].replace(status_mapping)

# Create columns for Good Debt and Bad Debt
credit_df["Good_Debt"] = np.where(credit_df["STATUS2"] == "Good Debt", 1, 0)
credit_df["Bad_Debt"] = np.where(credit_df["STATUS2"] == "Bad Debt", 1, 0)

Step 10: Group credit_record.csv by ID

In [37]:
# Group by ID and calculate total Good Debt and Bad Debt
credit_summary = credit_df.groupby("ID").agg(
    Total_Bad_Debt=("Bad_Debt", "sum"),
    Total_Good_Debt=("Good_Debt", "sum")
).reset_index()

 Step 11: Group application_record.csv by ID

In [38]:
# Group by ID and aggregate key fields
application_summary = application_df.groupby("ID").agg({
    "CODE_GENDER": "first",
    "CNT_CHILDREN": "first",
    "AMT_INCOME_TOTAL": "sum",
    "NAME_INCOME_TYPE": "first",
    "NAME_EDUCATION_TYPE": "first",
    "NAME_FAMILY_STATUS": "first",
    "NAME_HOUSING_TYPE": "first",
    "FLAG_OWN_CAR": "first",
    "FLAG_OWN_REALTY": "first",
    "FLAG_MOBIL": "first",
    "FLAG_WORK_PHONE": "first",
    "FLAG_PHONE": "first",
    "FLAG_EMAIL": "first",
    "OCCUPATION_TYPE": "first",
    "CNT_FAM_MEMBERS": "sum",
    "DAYS_BIRTH": "first",
    "DAYS_EMPLOYED": "first"
}).reset_index()

Step 12: Merge Processed Data

In [39]:
# Merge application data with aggregated data
application_df = pd.merge(filtered_application_df, application_summary, on="ID", how="left")

# Merge credit summary with application data
final_df = pd.merge(application_df, credit_summary, on="ID", how="left")

Step 13: Fill Missing Values

In [40]:
# Fill NaN values in Total_Bad_Debt and Total_Good_Debt with 0
final_df["Total_Bad_Debt"].fillna(0, inplace=True)
final_df["Total_Good_Debt"].fillna(0, inplace=True)

Step 14: Calculate Status Column

In [41]:
# Add Status column (1 if Good Debt > Bad Debt, else 0)
final_df["Status"] = np.where(final_df["Total_Good_Debt"] > final_df["Total_Bad_Debt"], 1, 0)

Step 15: Select grouped and cleaned 

In [42]:
# Drop the `_x` columns since `_y` has the cleaned grouped values
final_df = final_df.drop(columns=[col for col in final_df.columns if col.endswith('_x')])

# Rename `_y` columns to their original names
final_df = final_df.rename(columns=lambda col: col.replace('_y', '') if col.endswith('_y') else col)

# Print updated columns to verify
print("Updated Columns in final_df:")
print(final_df.columns.tolist())

Updated Columns in final_df:
['ID', 'Current_Date', 'Applicant_Age', 'Years_of_Working', 'CODE_GENDER', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'Total_Bad_Debt', 'Total_Good_Debt', 'Status']


Step 16: Drop Negative Years_of_Working Values

In [43]:
# Drop rows where Years_of_Working is negative
final_df = final_df[final_df["Years_of_Working"] >= 0]

# Print remaining records
print(f"✅ Remaining records after filtering: {len(final_df)}")

✅ Remaining records after filtering: 363227


Step 17: Rename and Select Columns

In [46]:
# Rename columns to match expected names
final_df.rename(columns={
    "ID":"Applicant_ID",
    "CODE_GENDER": "Applicant_Gender",
    "FLAG_OWN_CAR": "Owned_Car",
    "FLAG_OWN_REALTY": "Owned_Realty",
    "CNT_CHILDREN": "Total_Children",
    "AMT_INCOME_TOTAL": "Total_Income",
    "NAME_INCOME_TYPE": "Income_Type",
    "NAME_EDUCATION_TYPE": "Education_Type",
    "NAME_FAMILY_STATUS": "Family_Status",
    "NAME_HOUSING_TYPE": "Housing_Type",
    "CNT_FAM_MEMBERS": "Total_Family_Members"
}, inplace=True)

print("✅ Columns successfully renamed!")
print("Updated Columns in final_df:", final_df.columns.tolist())  # Verify

✅ Columns successfully renamed!
Updated Columns in final_df: ['Applicant_ID', 'Current_Date', 'Applicant_Age', 'Years_of_Working', 'Applicant_Gender', 'Total_Children', 'Total_Income', 'Income_Type', 'Education_Type', 'Family_Status', 'Housing_Type', 'Owned_Car', 'Owned_Realty', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'Total_Family_Members', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'Total_Bad_Debt', 'Total_Good_Debt', 'Status']


Step 17-19: Final Processing

In [47]:
# Select only the required columns
final_output = final_df[[
    "Applicant_ID", "Applicant_Gender", "Owned_Car", "Owned_Realty",
    "Total_Children", "Total_Income", "Income_Type", "Education_Type",
    "Family_Status", "Housing_Type", "Total_Family_Members",
    "Applicant_Age", "Years_of_Working", "Total_Bad_Debt", "Total_Good_Debt", "Status"
]]


Step 20: Save Final Output

In [48]:
# Save the final output to CSV
final_output.to_csv("Application_Data.csv", index=False, encoding="windows-1252")
print("\nETL process completed! Data saved to 'Application_Data.csv'.")


ETL process completed! Data saved to 'Application_Data.csv'.
